In [360]:
import pandas as pd
import numpy as np
from metar import Metar
import datetime as dt
import fuzzymatcher
import os
from geopy import distance
from geopy import Point
import glob
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from dateutil.parser import parse as parse_date
pd.set_option('display.max_columns', 999)

In [361]:
# load data frame
fw = pd.read_csv("..//Datasets/NEW/FW_with_airports.csv")
fw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,05:44PM CST,0:46,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,02:27PM CST,0:51,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,08:53AM CST,0:32,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,02:49PM CST,0:33,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,08:32AM CST,0:32,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND


In [362]:
fw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23160 entries, 0 to 23159
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tail_number               23160 non-null  object 
 1   date                      23160 non-null  object 
 2   aircraft                  22966 non-null  object 
 3   origin                    23160 non-null  object 
 4   origin_location           23160 non-null  object 
 5   destination               23159 non-null  object 
 6   destination_location      23159 non-null  object 
 7   departure                 23160 non-null  object 
 8   arrival                   22734 non-null  object 
 9   duration                  22711 non-null  object 
 10  Owner                     23160 non-null  object 
 11  origin_Latitude           23001 non-null  float64
 12  origin_Longitude          23001 non-null  float64
 13  destination_Latitude      23002 non-null  float64
 14  destin

In [363]:
fw.shape

(23160, 29)

In [367]:
# calculate the distance
for i in range(len(fw)):
    try:
        p1 = Point(str(fw.origin_Latitude[i]) + ' ' + str(fw.origin_Longitude[i]))
        p2 = Point(str(fw.destination_Latitude[i]) + ' ' + str(fw.destination_Longitude[i]))
    
        fw.loc[i, 'distance_mi'] = distance.distance(p1,p2).miles
    except:
        pass

In [368]:
# convert date to datetime
fw.date = pd.to_datetime(fw.date, format='%d-%b-%y')

In [369]:
# split time zone and assign to new variable for departure
fw['dep_time_zone'] = fw.departure.str.split(" ", n = 1, expand = True)[1]

# remove time zome from departure
fw.departure = fw.departure.str.split(" ", n = 1, expand = True)[0]

# split time zone and assign to new variable for arrival
fw['arr_time_zone'] = fw.arrival.str.split(" ", n = 1, expand = True)[1].str.split(" ", n = 1, expand = True)[0]

# remove time zome from arrival
fw.arrival = fw.arrival.str.split(" ", n = 1, expand = True)[0]

In [370]:
# rearrange data for the convenience 
fw = fw.iloc[:,:8].join([fw.iloc[:,-2], fw.iloc[:,8], fw.iloc[:,-1], fw.iloc[:,9], fw.iloc[:,-3], fw.iloc[:,10:29]])

In [371]:
# convert departure time to datetime format
fw.departure = pd.to_datetime(fw.departure, errors='coerce').dt.time

# convert arrival time to datetime format
fw.arrival = pd.to_datetime(fw.arrival, errors='coerce').dt.time

# drop null values
fw = fw.dropna(subset=['departure', 'arrival'])

#### Time zone differences
- **PST**: UTC−08, **CST**: UTC−06, **CAT**: UTC+02 
- **MST**: UTC−07, **AST**: UTC−04, **EST**: UTC−05
- **HST**: UTC−10, **NST**: UTC−03:30, **WAT**: UTC+01   
- **EET**: UTC+02, **EAT**: UTC+03, **AEDT**: UTC+11     
- **CE**T: UTC+01, **AKST**: UTC−09, **JST**: UTC+09, **ChST**: UTC+10  

In [372]:
# identify unique time zones for departure time
print(fw.dep_time_zone.sort_values(ascending=False).unique())

['WAT' 'PST' 'MST' 'HST' 'GMT' 'EST' 'EET' 'ChST' 'CST' 'AST' 'AKST'
 'AEDT' '-01' '+12' '+10' '+09' '+04' '+03']


In [373]:
# convert departure time to UTC
fw['dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'PST','departure']\
                       .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=8)).time())

fw.loc[fw.dep_time_zone == 'WAT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'WAT','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.dep_time_zone == 'ChST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'ChST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=10)).time())

fw.loc[fw.dep_time_zone == 'MST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'MST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=7)).time())

# fw.loc[fw.dep_time_zone == 'AEDT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AEDT','departure']\
#                                                      .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=11)).time())

fw.loc[fw.dep_time_zone == 'HST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'HST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=10)).time())

fw.loc[fw.dep_time_zone == 'GMT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'GMT','departure']

fw.loc[fw.dep_time_zone == 'EST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=5)).time())

fw.loc[fw.dep_time_zone == 'EET','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EET','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

fw.loc[fw.dep_time_zone == 'EAT','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'EAT','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=3)).time())

fw.loc[fw.dep_time_zone == 'CST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'CST','departure']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=6)).time())
# fw.loc[fw.dep_time_zone == 'CET','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'CET','departure']\
#                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.dep_time_zone == 'AST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AST','departure']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=4)).time())

fw.loc[fw.dep_time_zone == 'AKST','dep_UTC_time'] = fw.loc[fw.dep_time_zone == 'AKST','departure']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=9)).time())

In [374]:
# identify unique time zones for arrival time
print(fw.arr_time_zone.sort_values(ascending=False).unique())

['WAT' 'PST' 'MST' 'HST' 'GMT' 'EST' 'EET' 'ChST' 'CST' 'AST' 'AKST'
 'AEDT' '-01' '+12' '+10' '+09' '+06' '+04' '+03']


In [375]:
# convert arrival time to UTC
fw['arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'PST','arrival']\
                       .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=8)).time())

fw.loc[fw.arr_time_zone == 'NST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'NST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=3, minutes=30)).time())

fw.loc[fw.arr_time_zone == 'MST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'MST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=7)).time())

fw.loc[fw.arr_time_zone == 'WAT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'WAT','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=1)).time())

fw.loc[fw.arr_time_zone == 'HST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'HST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=10)).time())

fw.loc[fw.arr_time_zone == 'GMT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'GMT','arrival']

fw.loc[fw.arr_time_zone == 'EST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'EST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=5)).time())

fw.loc[fw.arr_time_zone == 'EET','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'EET','arrival']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

# fw.loc[fw.arr_time_zone == 'AEDT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AEDT','arrival']\
#                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=11)).time())

fw.loc[fw.arr_time_zone == 'CST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'CST','arrival']\
                                                     .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=6)).time())

fw.loc[fw.arr_time_zone == 'ChST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'ChST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=10)).time())

fw.loc[fw.arr_time_zone == 'CAT','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'CAT','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) - dt.timedelta(hours=2)).time())

fw.loc[fw.arr_time_zone == 'AST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AST','arrival']\
                                                  .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=4)).time())

fw.loc[fw.arr_time_zone == 'AKST','arr_UTC_time'] = fw.loc[fw.arr_time_zone == 'AKST','arrival']\
                                                   .apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=9)).time())

In [376]:
# # =======================================================================================
# #                       METAR data merging preparation for Origin
# # =======================================================================================
fw_origin = fw.loc[:,['tail_number','ICAO_code_origin', 'date', 'dep_UTC_time']]
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time
0,N100KB,KXWA,2021-02-20,22:57:00
1,N100KB,KBIS,2021-02-20,19:36:00
2,N100KB,KMOT,2021-02-18,14:20:00
3,N100KB,KXWA,2021-02-15,20:15:00
4,N100KB,KMOT,2021-02-15,14:00:00


In [378]:
# drop null values
fw_origin = fw_origin.dropna()

In [379]:
# sort by tail number 
fw_origin = fw_origin.sort_values('tail_number').reset_index(drop=True)

In [380]:
# extract hours
fw_origin['dep_UTC_hour'] = pd.to_datetime(fw_origin.dep_UTC_time.astype(str)).dt.hour

# extract minutes
fw_origin['dep_UTC_min'] = pd.to_datetime(fw_origin.dep_UTC_time.astype(str)).dt.minute

In [381]:
fw_origin.shape

(21845, 6)

In [382]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,dep_UTC_hour,dep_UTC_min
0,N100KB,KXWA,2021-02-20,22:57:00,22,57
1,N100KB,KXWA,2020-12-14,20:46:00,20,46
2,N100KB,KMOT,2020-12-20,14:48:00,14,48
3,N100KB,KFAR,2020-12-20,17:11:00,17,11
4,N100KB,KMOT,2020-12-20,01:40:00,1,40


In [239]:
# # =======================================================================================
# #                               METAR data scraping for Origin
# # =======================================================================================

# # create containers
# tail_number = []
# ICAO_origin = []
# date = []
# METAR = []

# for j in range(0,len(fw_origin)-1):
#     url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[j] \
#       + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[j]) \
#       + "&mes=" + str(fw_origin.date.dt.month.iloc[j]) + "&day=" + str(fw_origin.date.dt.day.iloc[j]) \
#       +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j]) + "&anof=" + str(fw_origin.date.dt.year.iloc[j]) \
#       + "&mesf="+ str(fw_origin.date.dt.month.iloc[j]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[j]) \
#     + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j] + 1) + "&minf=59&send=send"

#     # BeautifulSoup Magic
#     uClient = uReq(url)
#     page_html = uClient.read()
#     uClient.close()
#     page_soup = soup(page_html, 'html.parser')
    
#     if fw_origin.tail_number.iloc[j] == fw_origin.tail_number.iloc[j+1]:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_origin.tail_number.iloc[j]]
#             ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]

# # # if reaches the end
# #     if j == len(fw_origin)-2:
# #         url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[-1] \
# #           + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[-1]) \
# #           + "&mes=" + str(fw_origin.date.dt.month.iloc[-1]) + "&day=" + str(fw_origin.date.dt.day.iloc[-1]) \
# #           +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1]) + "&anof=" + str(fw_origin.date.dt.year.iloc[-1]) \
# #           + "&mesf="+ str(fw_origin.date.dt.month.iloc[-1]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[-1]) \
# #           + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1] + 1) + "&minf=59&send=send"
        
# #         # BeautifulSoup Magic
# #         uClient = uReq(url)
# #         page_html = uClient.read()
# #         uClient.close()
# #         page_soup = soup(page_html, 'html.parser')
        
# #         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
# #             tail_number += [fw_origin.tail_number.iloc[-1]]
# #             ICAO_origin += [fw_origin.ICAO_code_origin.iloc[-1]]
# #             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
# #             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
# #         pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])\
# #         .to_csv('../Datasets/METAR/FW/origin/' + fw_origin.tail_number.iloc[-1] + '.csv', index=False) 
        
#     else:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_origin.tail_number.iloc[j]]
#             ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
#         pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])\
#         .to_csv('../Datasets/METAR/FW/origin/' + fw_origin.tail_number.iloc[j] + '.csv', index=False) 
        
#         tail_number = []
#         ICAO_origin = []
#         date = []
#         METAR = []

In [240]:
# # concat all files
# all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv("FW_METAR_origin.csv", index=False, encoding='utf-8-sig')

In [390]:
# # =======================================================================================
# #                               METAR data merging for Origin
# # =======================================================================================
# load the origin METAR dataframe
metar_origin = pd.read_csv("../Datasets/METAR/FW/origin/FW_METAR_origin.csv")

In [391]:
metar_origin.head()

,tail_number,ICAO_origin,date,METAR
0,N783AL,KCOS,04/01/2021 19:54,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...
1,N783AL,KCOS,04/01/2021 18:54,METAR KCOS 041854Z 13008KT 10SM SCT250 04/M08 ...
2,N783AL,KAPA,04/01/2021 17:53,METAR KAPA 041753Z 22010KT 10SM FEW100 FEW200 ...
3,N783AL,KAPA,04/01/2021 16:53,METAR KAPA 041653Z 23009KT 10SM FEW100 FEW200 ...
4,N783AL,KSBS,01/01/2021 22:55,METAR KSBS 012255Z AUTO 00000KT 10SM CLR M04/M...


In [392]:
# split time from the date and assign a new column
metar_origin['dep_hour'] = metar_origin.date.str.split(" ", n=1, expand=True)[1]

# conver time to datetime and extract hour
metar_origin['dep_min'] = pd.to_datetime(metar_origin.dep_hour, format='%H:%M').dt.minute

# conver time to datetime and extract hour
metar_origin.dep_hour = pd.to_datetime(metar_origin.dep_hour, format='%H:%M').dt.hour

# remove time from the date columns
metar_origin.date = metar_origin.date.str.split(" ", n=1, expand=True)[0]

# convert date to datetime
metar_origin.date  = pd.to_datetime(metar_origin.date, format='%d/%m/%Y')

In [393]:
metar_origin.head()

,tail_number,ICAO_origin,date,METAR,dep_hour,dep_min
0,N783AL,KCOS,2021-01-04,METAR KCOS 041954Z 19007KT 10SM SCT250 08/M09 ...,19,54
1,N783AL,KCOS,2021-01-04,METAR KCOS 041854Z 13008KT 10SM SCT250 04/M08 ...,18,54
2,N783AL,KAPA,2021-01-04,METAR KAPA 041753Z 22010KT 10SM FEW100 FEW200 ...,17,53
3,N783AL,KAPA,2021-01-04,METAR KAPA 041653Z 23009KT 10SM FEW100 FEW200 ...,16,53
4,N783AL,KSBS,2021-01-01,METAR KSBS 012255Z AUTO 00000KT 10SM CLR M04/M...,22,55


In [396]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,dep_UTC_hour,dep_UTC_min
0,N100KB,KXWA,2021-02-20,22:57:00,22,57
1,N100KB,KXWA,2020-12-14,20:46:00,20,46
2,N100KB,KMOT,2020-12-20,14:48:00,14,48
3,N100KB,KFAR,2020-12-20,17:11:00,17,11
4,N100KB,KMOT,2020-12-20,01:40:00,1,40


In [400]:
fw_origin = fuzzymatcher.fuzzy_left_join(fw_origin, metar_origin.add_suffix('_origin'), left_on = ['tail_number','ICAO_code_origin', 'date', 'dep_UTC_hour', 'dep_UTC_min'], 
                                   right_on = ['tail_number_origin','ICAO_origin_origin', 'date_origin', 'dep_hour_origin', 'dep_min_origin']).iloc[:,3:]

In [401]:
fw_origin.shape

(21845, 12)

In [406]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,dep_UTC_hour,dep_UTC_min,tail_number_origin,ICAO_origin_origin,date_origin,METAR_origin,dep_hour_origin,dep_min_origin
0,N100KB,KXWA,2021-02-20,22:57:00,22,57,N100KB,KXWA,2021-02-20,METAR KXWA 202251Z AUTO 26013KT 10SM CLR 04/M0...,22,51
50,N100KB,KXWA,2020-12-14,20:46:00,20,46,N100KB,KXWA,2020-12-14,METAR KXWA 142051Z AUTO 16013G23KT 10SM FEW011...,20,51
100,N100KB,KMOT,2020-12-20,14:48:00,14,48,N100KB,KMOT,2020-12-20,METAR KMOT 201454Z 14007KT 10SM OVC095 M02/M06...,14,54
150,N100KB,KFAR,2020-12-20,17:11:00,17,11,N100KB,KFAR,2020-12-20,METAR KFAR 201753Z 17004KT 10SM CLR M02/M05 A2...,17,53
200,N100KB,KMOT,2020-12-20,01:40:00,1,40,N100KB,KMOT,2020-12-20,METAR KMOT 200154Z 25018G28KT 10SM CLR 04/M03 ...,1,54


In [409]:
# number of mismatched records
fw_origin.loc[(fw_origin.date != fw_origin.date_origin) | (fw_origin.tail_number != fw_origin.tail_number_origin)
              | (fw_origin.ICAO_code_origin != fw_origin.ICAO_origin_origin) | (fw_origin.dep_UTC_hour != fw_origin.dep_hour_origin)].shape

(2225, 12)

In [411]:
fw_origin.loc[(fw_origin.date != fw_origin.date_origin) | (fw_origin.tail_number != fw_origin.tail_number_origin)
              | (fw_origin.ICAO_code_origin != fw_origin.ICAO_origin_origin) | (fw_origin.dep_UTC_hour != fw_origin.dep_hour_origin)] = np.nan

In [413]:
fw_origin = fw_origin.dropna()

In [414]:
# drop duplicated columns
fw_origin = fw_origin.drop(['ICAO_origin_origin', 'dep_hour_origin', 'date_origin', 'dep_min_origin', 'tail_number_origin'], axis=1)

In [415]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,dep_UTC_hour,dep_UTC_min,METAR_origin
0,N100KB,KXWA,2021-02-20,22:57:00,22.0,57.0,METAR KXWA 202251Z AUTO 26013KT 10SM CLR 04/M0...
50,N100KB,KXWA,2020-12-14,20:46:00,20.0,46.0,METAR KXWA 142051Z AUTO 16013G23KT 10SM FEW011...
100,N100KB,KMOT,2020-12-20,14:48:00,14.0,48.0,METAR KMOT 201454Z 14007KT 10SM OVC095 M02/M06...
150,N100KB,KFAR,2020-12-20,17:11:00,17.0,11.0,METAR KFAR 201753Z 17004KT 10SM CLR M02/M05 A2...
200,N100KB,KMOT,2020-12-20,01:40:00,1.0,40.0,METAR KMOT 200154Z 25018G28KT 10SM CLR 04/M03 ...


In [416]:
# Decode origin METAR data 
for i in range(len(fw_origin)):
    try:
        code = fw_origin.METAR_origin.iloc[i]
        obs = Metar.Metar(code)
        # The 'time' attribute is a datetime object
        fw_origin.loc[i, 'METAR_time_origin'] = obs.time.ctime()
        # The report type attribute
        fw_origin.loc[i, 'report_type_origin'] = obs.report_type()
        # The 'temp' and 'dewpt' attributes are temperature objects
        fw_origin.loc[i, 'temperature_origin'] = obs.temp.string("F")
        fw_origin.loc[i, 'dew_point_origin'] = obs.dewpt.string("F")
        # The wind() method returns a string describing wind observations
        # which may include speed, direction, variability and gusts.
        fw_origin.loc[i, 'wind_origin'] = obs.wind()
        # The peak_wind() method returns a string describing the peak wind
        # speed and direction.
        fw_origin.loc[i, 'wind_peak_origin'] = obs.peak_wind()
        # The visibility() method summarizes the visibility observation.
        fw_origin.loc[i, 'visibility_origin'] = obs.visibility()
        # The 'press' attribute is a pressure object.
        fw_origin.loc[i, 'pressure_origin'] = obs.press.string("mb")
        # The 'press' attribute is a pressure object.
        fw_origin.loc[i, 'press_sea_level_origin'] = obs.press_sea_level.string("mb")
        # The sky_conditions() method summarizes the cloud-cover observations.
        fw_origin.loc[i, 'sky_origin'] = obs.sky_conditions()
        # The remarks() method describes the remark groups that were parsed
        fw_origin.loc[i, 'remarks_origin'] = obs.remarks()
    except:
        pass

In [417]:
# convert METAR_time_origin datetime
fw_origin.METAR_time_origin = pd.to_datetime(fw_origin.METAR_time_origin, format='%a %b %d %H:%M:%S %Y').dt.time

In [418]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time,dep_UTC_hour,dep_UTC_min,METAR_origin,METAR_time_origin,report_type_origin,temperature_origin,dew_point_origin,wind_origin,wind_peak_origin,visibility_origin,pressure_origin,press_sea_level_origin,sky_origin,remarks_origin
0,N100KB,KXWA,2021-02-20,22:57:00,22.0,57.0,METAR KXWA 202251Z AUTO 26013KT 10SM CLR 04/M0...,22:51:00,"routine report, cycle 23 (automatic report)",39.9 F,19.0 F,W at 13 knots,missing,10 miles,1008.5 mb,1012.1 mb,clear,Automated station (type 2)
50,N100KB,KXWA,2020-12-14,20:46:00,20.0,46.0,METAR KXWA 142051Z AUTO 16013G23KT 10SM FEW011...,21:51:00,"routine report, cycle 22 (automatic report)",25.0 F,12.9 F,SSW at 13 knots,missing,10 miles,1019.6 mb,1024.4 mb,clear,Automated station (type 2)
100,N100KB,KMOT,2020-12-20,14:48:00,14.0,48.0,METAR KMOT 201454Z 14007KT 10SM OVC095 M02/M06...,21:54:00,"routine report, cycle 22 (automatic report)",35.1 F,25.0 F,"W at 14 knots, gusting to 23 knots",missing,10 miles,1019.3 mb,1020.1 mb,a few clouds at 2800 feet; overcast at 4200 feet,Automated station (type 2)
150,N100KB,KFAR,2020-12-20,17:11:00,17.0,11.0,METAR KFAR 201753Z 17004KT 10SM CLR M02/M05 A2...,22:51:00,"routine report, cycle 23 (automatic report)",41.0 F,36.0 F,SW at 4 knots,missing,10 miles,1012.5 mb,1012.9 mb,a few clouds at 4500 feet,Automated station (type 2)
200,N100KB,KMOT,2020-12-20,01:40:00,1.0,40.0,METAR KMOT 200154Z 25018G28KT 10SM CLR 04/M03 ...,03:51:00,"routine report, cycle 4 (automatic report)",26.1 F,10.0 F,N at 6 knots,missing,10 miles,1017.6 mb,1017.7 mb,clear,Automated station (type 2)


In [359]:
fw_origin.to_csv('..//Datasets/NEW/fw_origin.csv', index=False)

In [ ]:
# # convert to string fot the best matching
# metar_origin.date = metar_origin.date.astype(str)
# metar_origin.departure = metar_origin.departure.astype(str)

# fw.date = fw.date.astype(str)
# fw.dep_UTC_time = fw.dep_UTC_time.astype(str)

In [ ]:
# metar_origin.head()

In [ ]:
# # merge origin METAR data 
# fw = fuzzymatcher.fuzzy_left_join(fw, metar_origin.add_suffix('_origin'), left_on = ['tail_number','ICAO_code_origin', 'date', 'dep_UTC_time'], 
#                                   right_on = ['tail_number_origin','ICAO_origin_origin', 'date_origin', 'departure_origin']).iloc[:,3:]

In [ ]:
# # count of records where the ICAO_code_origin is null or not in metar_origin column
# fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna())].shape

In [ ]:
# # Observe where the ICAO_code_origin is null or not in metar_origin columns
# fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna()), ['ICAO_code_origin', 'ICAO_origin_origin']].head()

In [ ]:
# # Drop where the ICAO_code_origin is null or not in metar_origin
# fw.loc[(~fw.ICAO_code_origin.isin(fw.ICAO_origin_origin)) | (fw.ICAO_code_origin.isna()), fw.columns[-5:39]] = np.nan

In [ ]:
# # number of mismached tail numbers
# fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull())].shape

In [ ]:
# # inspect this records
# fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull()),
#        ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

In [ ]:
# # drop this records
# fw.loc[(fw.tail_number != fw.tail_number_origin) & (fw.tail_number_origin.notnull()), fw.columns[-5:39]] = np.nan

In [ ]:
# # number of mismached ICAO_origin_origin
# fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin)].shape

In [ ]:
# # inspect this records
# fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin),
#        ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

In [ ]:
# # drop this records
# fw.loc[(fw.ICAO_code_origin.notnull()) & (fw.ICAO_origin_origin.notnull()) & (fw.ICAO_code_origin != fw.ICAO_origin_origin), fw.columns[-5:39]] = np.nan

In [ ]:
# # number of mismached date
# fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull())].shape

In [ ]:
# # observe this records 
# fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull()),
#     ['tail_number', 'ICAO_code_origin', 'date', 'dep_UTC_time', 'tail_number_origin', 'ICAO_origin_origin', 'date_origin','departure_origin']].head()

In [ ]:
# # drop this records
# fw.loc[(fw.date != fw.date_origin) & (fw.date_origin.notnull()), fw.columns[-5:39]] = np.nan

In [ ]:
# # how much we lost
# fw.METAR_origin.isna().sum()

In [ ]:
# # drop duplicated columns
# fw = fw.drop(['tail_number_origin', 'ICAO_origin_origin', 'date_origin', 'departure_origin'], axis=1)

In [ ]:
# # reset the indexes
# fw = fw.reset_index(drop=True)

In [ ]:
# Decode origin METAR data 
for i in range(len(fw)):
    try:
        code = fw.METAR_origin.iloc[i]
        obs = Metar.Metar(code)
        # The 'time' attribute is a datetime object
        fw.loc[i, 'METAR_time_origin'] = obs.time.ctime()
        # The report type attribute
        fw.loc[i, 'report_type_origin'] = obs.report_type()
        # The 'temp' and 'dewpt' attributes are temperature objects
        fw.loc[i, 'temperature_origin'] = obs.temp.string("F")
        fw.loc[i, 'dew_point_origin'] = obs.dewpt.string("F")
        # The wind() method returns a string describing wind observations
        # which may include speed, direction, variability and gusts.
        fw.loc[i, 'wind_origin'] = obs.wind()
        # The peak_wind() method returns a string describing the peak wind
        # speed and direction.
        fw.loc[i, 'wind_peak_origin'] = obs.peak_wind()
        # The visibility() method summarizes the visibility observation.
        fw.loc[i, 'visibility_origin'] = obs.visibility()
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'pressure_origin'] = obs.press.string("mb")
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'press_sea_level_origin'] = obs.press_sea_level.string("mb")
        # The sky_conditions() method summarizes the cloud-cover observations.
        fw.loc[i, 'sky_origin'] = obs.sky_conditions()
        # The remarks() method describes the remark groups that were parsed
        fw.loc[i, 'remarks_origin'] = obs.remarks()
    except:
        pass

In [ ]:
# convert METAR_time_origin datetime
fw.METAR_time_origin = pd.to_datetime(fw.METAR_time_origin, format='%a %b %d %H:%M:%S %Y').dt.time

In [ ]:
fw.head()

In [ ]:
# # =======================================================================================
# #                       METAR data merging preparation for Destination
# # =======================================================================================
# prepare dataframe for merging
fw_destination = fw.loc[:,['tail_number','ICAO_code_destination', 'date', 'arr_UTC_time']]
fw_destination.head()

In [ ]:
# drop null values
fw_destination = fw_destination.dropna()

In [ ]:
# sort by tail number 
fw_destination = fw_destination.sort_values('tail_number').reset_index(drop=True)

In [ ]:
fw_destination.head()

In [ ]:
# # =======================================================================================
# #                               METAR data scraping for Destination
# # =======================================================================================

# # create containers
# tail_number = []
# ICAO_destination = []
# date = []
# METAR = []

# for j in range(0,len(fw_destination)-1):
#     url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_destination.ICAO_code_destination.iloc[j] \
#       + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_destination.date.dt.year.iloc[j]) \
#       + "&mes=" + str(fw_destination.date.dt.month.iloc[j]) + "&day=" + str(fw_destination.date.dt.day.iloc[j]) \
#       +"&hora=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[j]) + "&anof=" + str(fw_destination.date.dt.year.iloc[j]) \
#       + "&mesf="+ str(fw_destination.date.dt.month.iloc[j]) + "&dayf=" + str(fw_destination.date.dt.day.iloc[j]) \
#     + "&horaf=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[j] + 1) + "&minf=59&send=send"

#     # BeautifulSoup Magic
#     uClient = uReq(url)
#     page_html = uClient.read()
#     uClient.close()
#     page_soup = soup(page_html, 'html.parser')
    
#     if fw_destination.tail_number.iloc[j] == fw_destination.tail_number.iloc[j+1]:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[j]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]

#     # if reaches the end
#     if j == len(fw_destination)-2:
#         url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_destination.ICAO_code_destination.iloc[-1] \
#           + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_destination.date.dt.year.iloc[-1]) \
#           + "&mes=" + str(fw_destination.date.dt.month.iloc[-1]) + "&day=" + str(fw_destination.date.dt.day.iloc[-1]) \
#           +"&hora=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1]) + "&anof=" + str(fw_destination.date.dt.year.iloc[-1]) \
#           + "&mesf="+ str(fw_destination.date.dt.month.iloc[-1]) + "&dayf=" + str(fw_destination.date.dt.day.iloc[-1]) \
#           + "&horaf=" + str(pd.to_datetime(fw_destination.arr_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1] + 1) + "&minf=59&send=send"
        
#         # BeautifulSoup Magic
#         uClient = uReq(url)
#         page_html = uClient.read()
#         uClient.close()
#         page_soup = soup(page_html, 'html.parser')
        
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[-1]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[-1]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
#         pd.DataFrame(list(zip(tail_number, ICAO_destination, date, METAR)), columns= ['tail_number', 'ICAO_destination', 'date', 'METAR'])\
#         .to_csv('../Datasets/METAR/FW/destination/' + fw_destination.tail_number.iloc[-1] + '.csv', index=False) 
        
#     else:
#         for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
#             tail_number += [fw_destination.tail_number.iloc[j]]
#             ICAO_destination += [fw_destination.ICAO_code_destination.iloc[j]]
#             date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
#             METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
#         pd.DataFrame(list(zip(tail_number, ICAO_destination, date, METAR)), columns= ['tail_number', 'ICAO_destination', 'date', 'METAR'])\
#         .to_csv('../Datasets/METAR/FW/destination/' + fw_destination.tail_number.iloc[j] + '.csv', index=False) 
        
#         tail_number = []
#         ICAO_destination = []
#         date = []
#         METAR = []

In [ ]:
# # concat all files
# all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv("FW_METAR_destination.csv", index=False, encoding='utf-8-sig')

In [ ]:
# # =======================================================================================
# #                               METAR data merging for Destination
# # =======================================================================================
# load the destination METAR dataframe
metar_destination = pd.read_csv("..//Datasets/METAR/FW/destination/FW_METAR_destination.csv")

In [ ]:
metar_destination.head()

In [ ]:
# split time from the date and assign a new column
metar_destination['arrival'] = metar_destination.date.str.split(" ", n=1, expand=True)[1]

# remove time from the date columns
metar_destination.date = metar_destination.date.str.split(" ", n=1, expand=True)[0]

In [ ]:
# convert date to datetime
metar_destination.date  = pd.to_datetime(metar_destination.date, format='%d/%m/%Y')

# conver time to datetime
metar_destination.arrival = pd.to_datetime(metar_destination.arrival, format='%H:%M').dt.time

In [ ]:
metar_destination.head()

In [ ]:
# convert to string fot the best matching
metar_destination.date = metar_destination.date.astype(str)
metar_destination.arrival = metar_destination.arrival.astype(str)

In [ ]:
# merge origin METAR data 
fw = fuzzymatcher.fuzzy_left_join(fw, metar_destination.add_suffix('_destination'), left_on = ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time'], 
                                  right_on = ['tail_number_destination','ICAO_destination_destination','date_destination', 'arrival_destination']).iloc[:,3:]

In [ ]:
# count of records where the ICAO_code_destination is null or not in metar_destination column
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna())].shape

In [ ]:
# Observe where the ICAO_code_destination is null or not in metar_destination columns
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna()), ['ICAO_code_destination', 'ICAO_destination_destination']].head()

In [ ]:
# Drop these records
fw.loc[(~fw.ICAO_code_destination.isin(fw.ICAO_destination_destination)) | (fw.ICAO_code_destination.isna()), fw.columns[-5:52]] = np.nan

In [ ]:
# number of mismached tail numbers
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull())].shape

In [ ]:
# inspect this records
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull()),
       ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

In [ ]:
# Drop these records
fw.loc[(fw.tail_number != fw.tail_number_destination) & (fw.tail_number_destination.notnull()), fw.columns[-5:52]] = np.nan

In [ ]:
# number of mismached ICAO_destination_destination
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination)].shape

In [ ]:
# inspect this records
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination),
       ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

In [ ]:
# Drop these records
fw.loc[(fw.ICAO_code_destination.notnull()) & (fw.ICAO_destination_destination.notnull()) & (fw.ICAO_code_destination != fw.ICAO_destination_destination), fw.columns[-5:52]] = np.nan

In [ ]:
# number of mismached date
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull())].shape

In [ ]:
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull()),
        ['tail_number', 'ICAO_code_destination', 'date', 'arr_UTC_time', 'tail_number_destination', 'ICAO_destination_destination', 'date_destination','arrival_destination']].head()

In [ ]:
# Drop these records
fw.loc[(fw.date != fw.date_destination) & (fw.date_destination.notnull()), fw.columns[-5:52]] = np.nan

In [ ]:
# how much we lost
fw.METAR_destination.isna().sum()

In [ ]:
# drop duplicated columns
fw = fw.drop(['tail_number_destination', 'ICAO_destination_destination', 'date_destination', 'arrival_destination'], axis=1)

In [ ]:
# reset the indexes
fw = fw.reset_index(drop=True)

In [ ]:
# Decode destination METAR data 
for i in range(len(fw)):
    try:
        code = fw.METAR_destination.iloc[i]
        obs = Metar.Metar(code)
        # The 'time' attribute is a datetime object
        fw.loc[i, 'METAR_time_destination'] = obs.time.ctime()
        # The report type attribute
        fw.loc[i, 'report_type_destination'] = obs.report_type()
        # The 'temp' and 'dewpt' attributes are temperature objects
        fw.loc[i, 'temperature_destination'] = obs.temp.string("F")
        fw.loc[i, 'dew_point_destination'] = obs.dewpt.string("F")
        # The wind() method returns a string describing wind observations
        # which may include speed, direction, variability and gusts.
        fw.loc[i, 'wind_destination'] = obs.wind()
        # The peak_wind() method returns a string describing the peak wind
        # speed and direction.
        fw.loc[i, 'wind_peak_destination'] = obs.peak_wind()
        # The visibility() method summarizes the visibility observation.
        fw.loc[i, 'visibility_destination'] = obs.visibility()
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'pressure_destination'] = obs.press.string("mb")
        # The 'press' attribute is a pressure object.
        fw.loc[i, 'press_sea_level_destination'] = obs.press_sea_level.string("mb")
        # The sky_conditions() method summarizes the cloud-cover observations.
        fw.loc[i, 'sky_destination'] = obs.sky_conditions()
        # The remarks() method describes the remark groups that were parsed
        fw.loc[i, 'remarks_destination'] = obs.remarks()
    except:
        pass

In [ ]:
# convert METAR_time_origin datetime
fw.METAR_time_destination = pd.to_datetime(fw.METAR_time_destination, format='%a %b %d %H:%M:%S %Y').dt.time

In [ ]:
fw.head()

In [ ]:
fw.isna().sum()

In [ ]:
fw.to_csv('../Datasets/NEW/FW_with_metar.csv', index=False)